In [1]:
import sqlite3

import numpy as np
import pandas as pd
from pathlib import Path

## Constants

In [2]:
__file__ = %pwd
DATA = Path(__file__).parent / "data"
PRIORITIES = DATA / "Priorities"
DATABASE = DATA / "databases" / "exploration.db"
DB_INSERT = False

## Load the Data

In [3]:
pub_schools_columns = [
    "CDSCode", "NCESDist", "NCESSchool", "StatusType",
    "County", "District", "School", "Street", "StreetAbr",
    "City", "Zip", "State", "MailStreet", "MailStreetAbr",
    "MailCity", "MailZip", "MailState", "Phone", "PhoneExt",
    "FaxNumber", "Website", "OpenDate", "ClosedDate", "Charter",
    "CharterNum", "FundingType", "DOC", "DOCType", "SOC",
    "SOCType", "EdOpsCode", "EdOpsName", "EILCode", "EILName",
    "GSoffered", "GSserved", "Virtual", "Magnet", "YearRound",
    "FederalDFCDistrictID", "Latitude", "Longitude", "AdmFName",
    "AdmLName", "LastUpDate", "Multilingual"
]

In [4]:
p1 = pd.read_excel(PRIORITIES / "Pr12024.xlsx")
p1_23 = pd.read_excel(PRIORITIES / "Pr12023.xlsx")

p2 = pd.read_excel(PRIORITIES / "Pr22024.xlsx")
p2_23 = pd.read_excel(PRIORITIES / "Pr22023.xlsx")

p3 = pd.read_excel(PRIORITIES / "Pr32024.xlsx")
p3_23 = pd.read_excel(PRIORITIES / "Pr32023.xlsx")

p6 = pd.read_excel(PRIORITIES / "Pr62024.xlsx")
p6_23 = pd.read_excel(PRIORITIES / "Pr62023.xlsx")

p7 = pd.read_excel(PRIORITIES / "Pr72024.xlsx")
p7_23 = pd.read_excel(PRIORITIES / "Pr72023.xlsx")

p9 = pd.read_excel(PRIORITIES / "Pr92024.xlsx")
p9_23 = pd.read_excel(PRIORITIES / "Pr92023.xlsx")

p10 = pd.read_excel(PRIORITIES / "Pr102024.xlsx")
p10_23 = pd.read_excel(PRIORITIES / "Pr102023.xlsx")

pub_schools = pd.read_excel(DATA / "pubschls.xlsx", names=pub_schools_columns, skiprows=5)[:-1] # So many junk rows in this dataset
cde = pd.read_csv(DATA / "CensusDay" / "cdenroll2324-v2.txt", sep="\t")

## Data Processing

All datasets besides the `cdenroll2324-v2.txt` dataset use an amalgamated `CDSCode` rather than independent `CountyID`, etc. columns. As such, we should split this column into `county_id`, `district_id`, and `school_id` in order for ease of access to their respective names in `Counties`, `Districts`, and `Schools` tables. 

In [5]:
def get_countyCode(cds_code: int) -> int:
    county_digits = 1e12
    return int(cds_code / county_digits)

def get_districtCode(cds_code: int) -> int:
    district_digits = 1e7
    county_digits = 1e12
    return int((cds_code % county_digits) / district_digits)

def get_schoolCode(cds_code: int) -> int:
    district_digits = 1e7
    return int(cds_code % district_digits)

def cds_code_split(cds_code: int) -> pd.Series:
    return pd.Series([get_countyCode(cds_code), get_districtCode(cds_code), get_schoolCode(cds_code)])

#### TODO: Note that while Priority Six for the 2024 school year has NULL in the `summary` column, it is used by the 2023 school year and cannot be dropped. It may be worth looking into coallescing `summary1`, `summary2`, and `summary3` into the single `summary` column instead.

In [6]:
# 2023 Priority Six lacks the multiple summary columns that 2024 has.
p6_23["summary1"] = p6_23["summary2"] = p6_23["summary3"] = np.nan

# Reorganize columns to match 2024 year.
p6_23 = p6_23[["cdsCode", "lea", "priorityNumber", "summary", 
          "summary1", "summary2", "summary3",
          "countyPerformance", "additionalInfo", "meetingDate", "year"]]

In [7]:
p1_23.columns = p1.columns
p2_23.columns = p2.columns

p3_23.columns = p3.columns
p6_23.columns = p6.columns

p7_23.columns = p7.columns
p9_23.columns = p9.columns
p10_23.columns = p10.columns

In [8]:
p1[["CountyCode", "DistrictCode", "SchoolCode"]] = p1.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p1_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p1_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p2[["CountyCode", "DistrictCode", "SchoolCode"]] = p2.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p2_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p2_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p3[["CountyCode", "DistrictCode", "SchoolCode"]] = p3.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p3_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p3_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p6[["CountyCode", "DistrictCode", "SchoolCode"]] = p6.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p6_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p6_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p7[["CountyCode", "DistrictCode", "SchoolCode"]] = p7.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p7_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p7_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p9[["CountyCode", "DistrictCode", "SchoolCode"]] = p9.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p9_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p9_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

p10[["CountyCode", "DistrictCode", "SchoolCode"]] = p10.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
p10_23[["CountyCode", "DistrictCode", "SchoolCode"]] = p10_23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

pub_schools[["CountyCode", "DistrictCode", "SchoolCode"]] = pub_schools.apply(
    lambda row: cds_code_split(int(row["CDSCode"])), axis=1
)

#### Coalesce every school into a single dataframe

In [9]:
cde_school_codes = cde[["SchoolCode", "SchoolName"]].drop_duplicates()
cde_school_codes_unique = cde_school_codes[cde_school_codes["SchoolCode"].notnull()]

In [10]:
pub_schools.columns

Index(['CDSCode', 'NCESDist', 'NCESSchool', 'StatusType', 'County', 'District',
       'School', 'Street', 'StreetAbr', 'City', 'Zip', 'State', 'MailStreet',
       'MailStreetAbr', 'MailCity', 'MailZip', 'MailState', 'Phone',
       'PhoneExt', 'FaxNumber', 'Website', 'OpenDate', 'ClosedDate', 'Charter',
       'CharterNum', 'FundingType', 'DOC', 'DOCType', 'SOC', 'SOCType',
       'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved',
       'Virtual', 'Magnet', 'YearRound', 'FederalDFCDistrictID', 'Latitude',
       'Longitude', 'AdmFName', 'AdmLName', 'LastUpDate', 'Multilingual',
       'CountyCode', 'DistrictCode', 'SchoolCode'],
      dtype='object')

In [11]:
# Check to see what is missing
pub_school_codes = pub_schools[
    (~pub_schools["SchoolCode"].isin(cde_school_codes_unique["SchoolCode"])) & (pub_schools["StatusType"] == "Active")
][["SchoolCode", "School"]]
pub_school_codes_unique = pub_school_codes[pub_school_codes["SchoolCode"].notnull()].drop_duplicates()
pub_school_codes_unique.columns = ["SchoolCode", "SchoolName"]
pub_school_codes_unique

,SchoolCode,SchoolName
7,129403,Epic Charter
41,6106751,Alameda County Special Education
56,130120,Alameda Adult
64,138214,Alameda Unified Special Education
109,122804,Berkeley Special Education Preschool
...,...,...
18221,5730080,West Sacramento School for Independent Study
18258,5738703,Woodland Adult Education
18279,5790019,Yolo County ROP
18285,141440,Yuba County Adult Education


In [12]:
school_codes = pd.concat([cde_school_codes_unique, pub_school_codes_unique]).drop_duplicates()
school_codes

,SchoolCode,SchoolName
272,112607.0,Envision Academy for Arts & Technology
295,123968.0,Community School for Creative Education
316,124172.0,Yu Ming Charter
336,125567.0,Urban Montessori Charter
356,130401.0,Alameda County Juvenile Hall/Court
...,...,...
18221,5730080.0,West Sacramento School for Independent Study
18258,5738703.0,Woodland Adult Education
18279,5790019.0,Yolo County ROP
18285,141440.0,Yuba County Adult Education


#### Coalesce every district into a single dataframe

In [13]:
cde_district_codes = cde[["DistrictCode", "DistrictName"]].drop_duplicates()
cde_district_codes_unique = cde_district_codes[cde_district_codes["DistrictCode"].notnull()]

In [14]:
# Check to see what is missing
pub_district_codes = pub_schools[
    (~pub_schools["DistrictCode"].isin(cde_district_codes_unique["DistrictCode"])) & (pub_schools["StatusType"] == "Active")
][["DistrictCode", "District"]]
pub_district_codes_unique = pub_district_codes[pub_district_codes["DistrictCode"].notnull()].drop_duplicates()
pub_district_codes_unique.columns = ["DistrictCode", "DistrictName"]
pub_district_codes_unique

,DistrictCode,DistrictName
722,74005,Tri-Valley ROP
724,74013,Eden Area ROP
726,74021,Mission Valley ROC/P
845,74856,Amador County ROP
1018,74682,Butte County ROP
...,...,...
16814,74591,Sonoma County ROP
17097,74609,Yosemite ROP
17244,74633,Tri-County ROP
18149,74617,Ventura County ROP


In [15]:
district_codes = pd.concat([cde_district_codes_unique, pub_district_codes_unique]).drop_duplicates()
district_codes

,DistrictCode,DistrictName
189,10017.0,Alameda County Office of Education
556,31609.0,California School for the Blind (State Special...
619,31617.0,California School for the Deaf-Fremont (State ...
694,61119.0,Alameda Unified
1176,61127.0,Albany City Unified
...,...,...
16814,74591.0,Sonoma County ROP
17097,74609.0,Yosemite ROP
17244,74633.0,Tri-County ROP
18149,74617.0,Ventura County ROP


#### Coalesce every county into a single dataframe

In [16]:
cde_county_codes = cde[["CountyCode", "CountyName"]].drop_duplicates()
cde_county_codes_unique = cde_county_codes[cde_county_codes["CountyCode"].notnull()]

In [17]:
# Check to see what is missing
pub_county_codes = pub_schools[~pub_schools["CountyCode"].isin(cde_county_codes_unique["CountyCode"])][["CountyCode", "County"]]
pub_county_codes_unique = pub_county_codes[pub_county_codes["CountyCode"].notnull()].drop_duplicates()
pub_county_codes_unique.columns = ["CountyCode", "CountyName"]
pub_county_codes_unique

,CountyCode,CountyName


#### Coalescing is apparently not necessary here

In [18]:
county_codes = cde_county_codes_unique

#### Some of the `lea` values in the priorities dataset do not translate directly to `school_codes` or `district_codes`, as such, we need to keep them.

In [19]:
p1[(~p1["lea"].isin(school_codes["SchoolName"])) & (p1["SchoolCode"] != 0)]

,cdsCode,lea,priorityNumber,numMisassignments,numMaterials,numFacilities,countyPerformance,additionalInfo,meetingDate,year,CountyCode,DistrictCode,SchoolCode
476,19101990135582,Westbrook Academy,1,NaN,0,0.0,Met,By the end of the 2023-24 school year the scho...,2024-06-13,2024,19,10199,135582
621,19647330117846,Para Los Niños Middle,1,NaN,NaN,NaN,Not Met,NaN,NaN,2024,19,64733,117846
651,19647330122630,Para Los Niños - Evelyn Thurman Gratts Primary,1,NaN,NaN,NaN,Not Met,NaN,NaN,2024,19,64733,122630
824,19647336120489,Para Los Niños Charter,1,NaN,0,0.0,Not Met,NaN,NaN,2024,19,64733,6120489
917,20652430107938,Liberty Charter,1,NaN,0/0%,0.0,Met,"At Liberty Charter, we firmly believe that our...",2024-06-13,2024,20,65243,107938
965,23656150140814,Shanél Valley Academy,1,NaN,0/0%,0.0,Met,NaN,2024-06-27,2024,23,65615,140814
1079,30103060134239,Epic California Academy,1,NaN,0,0.0,Met,NaN,2024-06-06,2024,30,10306,134239
1099,30664640106765,California Online Public Schools Southern Cali...,1,NaN,0,0.0,Met,NaN,2024-06-04,2024,30,66464,106765
1432,37681060137034,Altus Schools North County,1,NaN,0,0.0,Met,NaN,2024-06-26,2024,37,68106,137034
1494,37683383730959,Altus Schools Charter School of San Diego,1,NaN,0,0.0,Met,NaN,2024-06-26,2024,37,68338,3730959


#### Verify if all `meetingDate`s are equivalent within schools

In [20]:
priorities = [
    p2[["cdsCode", "meetingDate"]],
    p3[["cdsCode", "meetingDate"]], 
    p6[["cdsCode", "meetingDate"]],
    p7[["cdsCode", "meetingDate"]],
    p9[["cdsCode", "meetingDate"]],
    p10[["cdsCode", "meetingDate"]]
]

priorities_23 = [
    p2_23[["cdsCode", "meetingDate"]],
    p3_23[["cdsCode", "meetingDate"]], 
    p6_23[["cdsCode", "meetingDate"]],
    p7_23[["cdsCode", "meetingDate"]],
    p9_23[["cdsCode", "meetingDate"]],
    p10_23[["cdsCode", "meetingDate"]]
]

priority_codes = p1[["cdsCode", "meetingDate"]]
priority_codes_23 = p1_23[["cdsCode", "meetingDate"]]

i = 0
for p in priorities:
    priority_codes = pd.merge(priority_codes, p, how="outer", on="cdsCode", suffixes=(f"_x{i}", f"_y{i}"))
    i += 1

i = 0
for p in priorities_23:
    priority_codes_23 = pd.merge(priority_codes_23, p, how="outer", on="cdsCode", suffixes=(f"_x{i}", f"_y{i}"))
    i += 1

In [21]:
relevant_columns = priority_codes.columns.to_list()
relevant_columns.remove("cdsCode")

relevant_columns_23 = priority_codes.columns.to_list()
relevant_columns_23.remove("cdsCode")
def is_equal(row) -> bool:
    for i in relevant_columns:
        for ii in relevant_columns:
            if pd.isna(row[i]) or pd.isna(row[ii]):
                continue
            if row[i] != row[ii]:
                print(f"{row[i]} != {row[ii]}")
                return False
    return True

In [22]:
priority_codes["equal"] = priority_codes.apply(lambda x: is_equal(x), axis=1)
priority_codes_23["equal"] = priority_codes_23.apply(lambda x: is_equal(x), axis=1)

In [23]:
priority_codes[priority_codes["equal"] != True]

,cdsCode,meetingDate_x0,meetingDate_y0,meetingDate_x2,meetingDate_y2,meetingDate_x4,meetingDate_y4,meetingDate,equal


In [24]:
priority_codes_23[priority_codes_23["equal"] != True]

,cdsCode,meetingDate_x0,meetingDate_y0,meetingDate_x2,meetingDate_y2,meetingDate_x4,meetingDate_y4,meetingDate,equal


They are! We can use a foreign key lookup utilizing `cdsCode` in a `MeetingDates` table.

In [25]:
p1_concat = pd.merge(p1[["cdsCode", "meetingDate"]], 
                     p1_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p2_concat = pd.merge(p2[["cdsCode", "meetingDate"]], 
                     p2_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p3_concat = pd.merge(p3[["cdsCode", "meetingDate"]], 
                     p3_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p6_concat = pd.merge(p6[["cdsCode", "meetingDate"]], 
                     p6_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p7_concat = pd.merge(p7[["cdsCode", "meetingDate"]], 
                     p7_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p9_concat = pd.merge(p9[["cdsCode", "meetingDate"]], 
                     p9_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")
p10_concat = pd.merge(p10[["cdsCode", "meetingDate"]], 
                     p10_23[["cdsCode", "meetingDate"]],
                     how="outer", on="cdsCode")

In [26]:
p1_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p2_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p3_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p6_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p7_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p9_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]
p10_concat.columns = ["cdsCode", "meetingDate2024", "meetingDate2023"]

In [27]:
priorities = [
    p1_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]],
    p2_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]],
    p3_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]], 
    p6_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]],
    p7_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]],
    p9_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]],
    p10_concat[["cdsCode", "meetingDate2023", "meetingDate2024"]]
]
meeting_dates = pd.concat([i for i in priorities]).drop_duplicates()
meeting_dates = meeting_dates[meeting_dates["cdsCode"].notnull()]

#### I can't see a reason to keep both `StreetAbr` and `Street` given this; drop the abbreviated version.

In [28]:
print(pub_schools[["StreetAbr", "Street"]].drop_duplicates())
print(pub_schools[["MailStreetAbr", "MailStreet"]].drop_duplicates())

                           StreetAbr                            Street
0               313 West Winton Ave.            313 West Winton Avenue
1      39899 Balentine Dr., Ste. 335  39899 Balentine Drive, Suite 335
2                   1515 Webster St.               1515 Webster Street
3                2125 Jefferson Ave.             2125 Jefferson Avenue
4           2111 International Blvd.      2111 International Boulevard
...                              ...                               ...
18351                 456 Beale Hwy.                 456 Beale Highway
18352                 123 Beale Hwy.                 123 Beale Highway
18353                 111 Hooper St.                 111 Hooper Street
18355             711 West Olive St.             711 West Olive Street
18357             1010 Wheatland Rd.               1010 Wheatland Road

[13994 rows x 2 columns]
                       MailStreetAbr                        MailStreet
0               313 West Winton Ave.            313

#### There are no non-null `numMisassignments`, as such, we can drop them!

In [29]:
print(p1[p1["numMisassignments"].notnull()])
print(p1_23[p1_23["numMisassignments"].notnull()])

Empty DataFrame
Columns: [cdsCode, lea, priorityNumber, numMisassignments, numMaterials, numFacilities, countyPerformance, additionalInfo, meetingDate, year, CountyCode, DistrictCode, SchoolCode]
Index: []
Empty DataFrame
Columns: [cdsCode, lea, priorityNumber, numMisassignments, numMaterials, numFacilities, countyPerformance, additionalInfo, meetingDate, year, CountyCode, DistrictCode, SchoolCode]
Index: []


## Upload to database

In [30]:
con = sqlite3.connect(DATABASE)
cur = con.cursor()

#### Drop all columns that are no longer necessary.

In [31]:
p_1 = pd.concat([p1, p1_23]).drop_duplicates()
p_2 = pd.concat([p2, p2_23]).drop_duplicates()
p_3 = pd.concat([p3, p3_23]).drop_duplicates()
p_6 = pd.concat([p6, p6_23]).drop_duplicates()
p_7 = pd.concat([p7, p7_23]).drop_duplicates()
p_9 = pd.concat([p9, p9_23]).drop_duplicates()
p_10 = pd.concat([p10, p10_23]).drop_duplicates()

In [32]:
p_1 = p_1[p_1.columns.difference(["priorityNumber", "numMisassignments", "meetingDate"])]
p_2 = p_2[p_2.columns.difference(["priorityNumber", "meetingDate"])]
p_3 = p_3[p_3.columns.difference(["priorityNumber", "meetingDate"])]
p_6 = p_6[p_6.columns.difference(["priorityNumber", "meetingDate"])]
p_7 = p_7[p_7.columns.difference(["priorityNumber", "meetingDate"])]
p_9 = p_9[p_9.columns.difference(["priorityNumber", "meetingDate"])]
p_10 = p_10[p_10.columns.difference(["priorityNumber", "meetingDate"])]

cde_dropped = cde[cde.columns.difference(["CountyName", "DistrictName", "SchoolName"])]
pub_schools_dropped = pub_schools[pub_schools.columns.difference([
    "County", "District", "School", "StreetAbr", "State", "MailStreetAbr", "MailState"
])]

In [33]:
if DB_INSERT: # Don't need to run this again!
    district_codes.to_sql("Districts", con, index=False, if_exists="append")
    school_codes.to_sql("Schools", con, index=False, if_exists="append")
    county_codes.to_sql("Counties", con, index=False, if_exists="append")
    meeting_dates.to_sql("MeetingDates", con, index=False, if_exists="append")

In [34]:
if DB_INSERT:
    p_1.to_sql("PriorityOne", con, index=False, if_exists="append")
    p_2.to_sql("PriorityTwo", con, index=False, if_exists="append")
    p_3.to_sql("PriorityThree", con, index=False, if_exists="append")
    p_6.to_sql("PrioritySix", con, index=False, if_exists="append")
    p_7.to_sql("PrioritySeven", con, index=False, if_exists="append")
    p_9.to_sql("PriorityNine", con, index=False, if_exists="append")
    p_10.to_sql("PriorityTen", con, index=False, if_exists="append")

In [35]:
if DB_INSERT:
    cde_dropped.to_sql("CensusDay", con, index=False, if_exists="append")
    pub_schools_dropped.to_sql("PublicSchools", con, index=False, if_exists="append")

In [36]:
con.close()

### ACGR Dataset

In [37]:
DB_INSERT_ACGR = False

In [38]:
acgr_columns = [
    "Year", "AggregateLevel", "CountyCode",
    "DistrictCode", "SchoolCode", "County",
    "District", "School", "CharterSchool", 
    "DASS", "ReportingCategory", "CohortStudents",
    "RegHSDiploma", "RegHSDiplomaRaet", "UniReqs",
    "UniReqsPercent", "Biliteracy", "BiliteracyRate",
    "Merit", "MeritRate", "Exemption",
    "ExemptionRate", "CPP", "CPPRate",
    "AdultEd", "AdultEdRate", "SPED",
    "SPEDRate", "GED", "GEDRate",
    "Other", "OtherRate", "Dropout",
    "DropoutRate", "StillEnrolled", "StillEnrolledRate"
]

In [39]:
# Load ACGR dataframes.
acgr24 = pd.read_csv(DATA / "ACGR" / "acgr24.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr23 = pd.read_csv(DATA / "ACGR" / "acgr23-v2.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr22 = pd.read_csv(DATA / "ACGR" / "acgr22-v3.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr21 = pd.read_csv(DATA / "ACGR" / "acgr21.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr20 = pd.read_csv(DATA / "ACGR" / "acgr20.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr19 = pd.read_csv(DATA / "ACGR" / "acgr19.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr18 = pd.read_csv(DATA / "ACGR" / "acgr18.txt", sep="\t", names=acgr_columns, skiprows=1)
acgr17 = pd.read_csv(DATA / "ACGR" / "acgr17.txt", sep="\t", names=acgr_columns, skiprows=1)

# Concat all ACGR into one dataframe
acgr = pd.concat([
    acgr24, acgr23,
    acgr22, acgr21,
    acgr20, acgr19,
    acgr18, acgr17
])

#### There are some schools that have not been accounted for in our `Schools` table yet; best to add them!

In [40]:
unaccounted_schools = acgr[~acgr["SchoolCode"].isin(school_codes["SchoolCode"])][["SchoolCode", "School"]].drop_duplicates()
unaccounted_schools

,SchoolCode,School
0,NaN,NaN
1486,132555.0,Conservatory of Vocal/Instrumental Arts High
4074,106625.0,Elk Creek
5596,1030766.0,Hallmark Charter
8972,132571.0,Konocti Education Center
...,...,...
110287,129742.0,Inspire Charter School
110615,134585.0,Pathways Academy Charter School Adult Education
132319,128363.0,College Prep High
180967,101980.0,Vallejo Education Academy


In [41]:
# Match column naming convention.
unaccounted_schools[["SchoolCode", "SchoolName"]] = unaccounted_schools[["SchoolCode", "School"]]
unaccounted_schools = unaccounted_schools[["SchoolCode", "SchoolName"]]

# Drop NaN row.
unaccounted_schools = unaccounted_schools.iloc[1:]

In [42]:
unaccounted_schools

,SchoolCode,SchoolName
1486,132555.0,Conservatory of Vocal/Instrumental Arts High
4074,106625.0,Elk Creek
5596,1030766.0,Hallmark Charter
8972,132571.0,Konocti Education Center
9321,100776.0,North Valley Military Institute College Prepar...
...,...,...
110287,129742.0,Inspire Charter School
110615,134585.0,Pathways Academy Charter School Adult Education
132319,128363.0,College Prep High
180967,101980.0,Vallejo Education Academy


#### Add cdsCode to the table.

In [43]:
def merge_codes(county_code: float, district_code: float, school_code: float) -> int:
    cds_code = 0
    if not np.isnan(county_code):
        cds_code += county_code * 1e12
    if not np.isnan(district_code):
        cds_code += district_code * 1e7
    if not np.isnan(school_code):
        cds_code += school_code
    return int(cds_code)

In [44]:
acgr["cdsCode"] = acgr.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
acgr[["cdsCode", "CountyCode", "DistrictCode", "SchoolCode"]]

,cdsCode,CountyCode,DistrictCode,SchoolCode
0,0,0,NaN,NaN
1,0,0,NaN,NaN
2,0,0,NaN,NaN
3,0,0,NaN,NaN
4,0,0,NaN,NaN
...,...,...,...,...
195864,0,0,NaN,NaN
195865,0,0,NaN,NaN
195866,0,0,NaN,NaN
195867,0,0,NaN,NaN


#### No longer necessary to hold onto `County`, `District`, or `School` given this information is in the `Counties`, `Districts`, and `Schools` tables respectively

In [45]:
acgr_cleaned = acgr[acgr.columns.difference(["County", "District", "School"])]
acgr_cleaned.columns

Index(['AdultEd', 'AdultEdRate', 'AggregateLevel', 'Biliteracy',
       'BiliteracyRate', 'CPP', 'CPPRate', 'CharterSchool', 'CohortStudents',
       'CountyCode', 'DASS', 'DistrictCode', 'Dropout', 'DropoutRate',
       'Exemption', 'ExemptionRate', 'GED', 'GEDRate', 'Merit', 'MeritRate',
       'Other', 'OtherRate', 'RegHSDiploma', 'RegHSDiplomaRaet',
       'ReportingCategory', 'SPED', 'SPEDRate', 'SchoolCode', 'StillEnrolled',
       'StillEnrolledRate', 'UniReqs', 'UniReqsPercent', 'Year', 'cdsCode'],
      dtype='object')

In [46]:
con = sqlite3.connect(DATABASE)
cur = con.cursor()

In [47]:
if DB_INSERT_ACGR:
    unaccounted_schools.to_sql("Schools", con, index=False, if_exists="append")
    acgr_cleaned.to_sql("ACGR", con, index=False, if_exists="append")

In [48]:
con.close()

### Student Enrollment Dataset

This dataset is formatted differently than the Census Day Enrollment Dataset however, it functions as that dataset for years before 2023-2024. As such, this requires some processing to get it into the correct structure.

In [49]:
enr20_22 = pd.read_csv(DATA / "CensusDay" / "enr202022-v2.txt", sep="\t")
enr17_19 = pd.read_csv(DATA / "CensusDay" / "enr201719-v2.txt", sep="\t")

DB_INSERT_SE = False

In [50]:
enr20_22

,ACADEMIC_YEAR,CDS_CODE,COUNTY,DISTRICT,SCHOOL,ENR_TYPE,RACE_ETHNICITY,GENDER,GR_KN,GR_1,...,GR_7,GR_8,UNGR_ELM,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL,ADULT
0,2020-21,1100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,0,F,0,0,...,3,0,0,1,0,1,0,0,6,0
1,2020-21,1100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,0,M,0,0,...,0,0,0,0,1,1,0,0,3,0
2,2020-21,1100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,1,F,0,0,...,0,0,0,1,1,0,1,0,3,0
3,2020-21,1100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,2,F,0,0,...,0,0,0,1,0,0,0,0,1,0
4,2020-21,1100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,2,M,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793375,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,F,0,0,...,0,0,0,35,54,60,43,0,192,0
793376,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,M,0,0,...,0,0,0,61,67,61,63,0,252,0
793377,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,X,0,0,...,0,0,0,0,0,1,0,0,1,0
793378,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,9,F,0,0,...,0,0,0,16,10,14,12,0,52,0


In [51]:
cde_dropped.columns

Index(['AcademicYear', 'AggregateLevel', 'Charter', 'CountyCode',
       'DistrictCode', 'GR_01', 'GR_02', 'GR_03', 'GR_04', 'GR_05', 'GR_06',
       'GR_07', 'GR_08', 'GR_09', 'GR_10', 'GR_11', 'GR_12', 'GR_KN', 'GR_TK',
       'ReportingCategory', 'SchoolCode', 'TOTAL_ENR'],
      dtype='object')

In [52]:
enr20_22.columns

Index(['ACADEMIC_YEAR', 'CDS_CODE', 'COUNTY', 'DISTRICT', 'SCHOOL', 'ENR_TYPE',
       'RACE_ETHNICITY', 'GENDER', 'GR_KN', 'GR_1', 'GR_2', 'GR_3', 'GR_4',
       'GR_5', 'GR_6', 'GR_7', 'GR_8', 'UNGR_ELM', 'GR_9', 'GR_10', 'GR_11',
       'GR_12', 'UNGR_SEC', 'ENR_TOTAL', 'ADULT'],
      dtype='object')

In [53]:
# Both datasets only aggregate at the School level.
enr20_22["AggregateLevel"] = "S"
enr17_19["AggregateLevel"] = "S"

# Split the cds_code into County, District, and School codes
enr20_22[["CountyCode", "DistrictCode", "SchoolCode"]] = enr20_22.apply(lambda row: cds_code_split(int(row["CDS_CODE"])), axis=1)
enr17_19[["CountyCode", "DistrictCode", "SchoolCode"]] = enr17_19.apply(lambda row: cds_code_split(int(row["CDS_CODE"])), axis=1)

# Drop columns that aren't reflected in CensusDay
enr20_22 = enr20_22[enr20_22.columns.difference([
    "CDS_CODE", "COUNTY", 
    "DISTRICT", "SCHOOL", 
    "ENR_TYPE", "UNGR_ELM",
    "UNGR_SEC", "ADULT"
])]

enr17_19 = enr17_19[enr17_19.columns.difference([
    "CDS_CODE", "COUNTY", 
    "DISTRICT", "SCHOOL", 
    "ENR_TYPE", "UNGR_ELM",
    "UNGR_SEC", "ADULT"
])]

enr20_22 = enr20_22.drop_duplicates()
enr17_19 = enr17_19.drop_duplicates()

#### Get marginals of each demographic in order to match `CensusDay`'s schema.

In [54]:
genders = ["M", "F", "Z"]
races = [i for i in range(0, 10)]
enr_cols = [
    "ENR_TOTAL", "GR_KN", "GR_1", "GR_2",
    "GR_3", "GR_4", "GR_5", "GR_6", "GR_7",
    "GR_8", "GR_9", "GR_10", "GR_11", "GR_12"
]

def get_marginals(df, enr: list):
    # Count based on gender.
    for gender in genders:
        df_filtered = df[df["GENDER"] == gender]
        if df_filtered.empty:
            continue
        row = {
            "AcademicYear": df_filtered["ACADEMIC_YEAR"].iloc[0],
            "AggregateLevel": "S",
            "Charter": None,
            "CountyCode": df_filtered["CountyCode"].iloc[0],
            "DistrictCode": df_filtered["DistrictCode"].iloc[0],
            "SchoolCode": df_filtered["SchoolCode"].iloc[0],
            "GR_TK": None,
            "ReportingCategory": gender
        }
        for enr_col in enr_cols:
            row[enr_col] = df_filtered[enr_col].sum()
        enr.append(row)

    # Count based on race.
    for race in races:
        df_filtered = df[df["RACE_ETHNICITY"] == race]
        if df_filtered.empty:
            continue
        row = {
            "AcademicYear": df_filtered["ACADEMIC_YEAR"].iloc[0],
            "AggregateLevel": "S",
            "Charter": None,
            "CountyCode": df_filtered["CountyCode"].iloc[0],
            "DistrictCode": df_filtered["DistrictCode"].iloc[0],
            "SchoolCode": df_filtered["SchoolCode"].iloc[0],
            "GR_TK": None,
            "ReportingCategory": race
        }
        for enr_col in enr_cols:
            row[enr_col] = df_filtered[enr_col].sum()
        enr.append(row)

    # Total Count
    row = {
        "AcademicYear": df["ACADEMIC_YEAR"].iloc[0],
        "AggregateLevel": "S",
        "Charter": None,
        "CountyCode": df["CountyCode"].iloc[0],
        "DistrictCode": df["DistrictCode"].iloc[0],
        "SchoolCode": df["SchoolCode"].iloc[0],
        "GR_TK": None,
        "ReportingCategory": "TA"
    }
    for enr_col in enr_cols:
        row[enr_col] = df[enr_col].sum()
    enr.append(row)

In [55]:
enr = []
enr20_22_grouped = enr20_22.groupby(["ACADEMIC_YEAR"])["SchoolCode"].unique()
enr17_19_grouped = enr17_19.groupby(["ACADEMIC_YEAR"])["SchoolCode"].unique()

for index in enr17_19_grouped.index:
    print(f"------ Gathering Marginals for year {index} ------")
    for school_code in enr17_19_grouped[index]:
        get_marginals(enr17_19[(enr17_19["ACADEMIC_YEAR"] == index) & (enr17_19["SchoolCode"] == school_code)], enr)

for index in enr20_22_grouped.index:
    print(f"------ Gathering Marginals for year {index} ------")
    for school_code in enr20_22_grouped[index]:
        get_marginals(enr20_22[(enr20_22["ACADEMIC_YEAR"] == index) & (enr20_22["SchoolCode"] == school_code)], enr)

enr_marginals = pd.DataFrame(enr)
enr_marginals

------ Gathering Marginals for year 2017-18 ------
------ Gathering Marginals for year 2018-19 ------
------ Gathering Marginals for year 2019-20 ------
------ Gathering Marginals for year 2020-21 ------
------ Gathering Marginals for year 2021-22 ------
------ Gathering Marginals for year 2022-23 ------


,AcademicYear,AggregateLevel,Charter,CountyCode,DistrictCode,SchoolCode,GR_TK,ReportingCategory,ENR_TOTAL,GR_KN,...,GR_3,GR_4,GR_5,GR_6,GR_7,GR_8,GR_9,GR_10,GR_11,GR_12
0,2017-18,S,None,1,10017,112607,None,M,193,0,...,0,0,0,0,0,0,49,53,56,35
1,2017-18,S,None,1,10017,112607,None,F,220,0,...,0,0,0,0,0,0,60,56,46,58
2,2017-18,S,None,1,10017,112607,None,1,6,0,...,0,0,0,0,0,0,1,1,1,3
3,2017-18,S,None,1,10017,112607,None,2,9,0,...,0,0,0,0,0,0,1,3,3,2
4,2017-18,S,None,1,10017,112607,None,3,6,0,...,0,0,0,0,0,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604259,2022-23,S,None,58,72769,5838305,None,5,338,0,...,0,0,0,0,0,0,103,85,81,69
604260,2022-23,S,None,58,72769,5838305,None,6,38,0,...,0,0,0,0,0,0,10,9,10,9
604261,2022-23,S,None,58,72769,5838305,None,7,445,0,...,0,0,0,0,0,0,96,121,122,106
604262,2022-23,S,None,58,72769,5838305,None,9,111,0,...,0,0,0,0,0,0,30,29,28,24


In [56]:
# Reorganize the column order
enr_marginals = enr_marginals[[
    "AcademicYear", "AggregateLevel", "Charter", "CountyCode",
    "DistrictCode", "GR_1", "GR_2", "GR_3", "GR_4", "GR_5", "GR_6",
    "GR_7", "GR_8", "GR_9", "GR_10", "GR_11", "GR_12", "GR_KN", "GR_TK",
    "ReportingCategory", "SchoolCode", "ENR_TOTAL"]]

# Match column names to cde_dropped
enr_marginals.columns = cde_dropped.columns
enr_marginals

,AcademicYear,AggregateLevel,Charter,CountyCode,DistrictCode,GR_01,GR_02,GR_03,GR_04,GR_05,...,GR_08,GR_09,GR_10,GR_11,GR_12,GR_KN,GR_TK,ReportingCategory,SchoolCode,TOTAL_ENR
0,2017-18,S,None,1,10017,0,0,0,0,0,...,0,49,53,56,35,0,None,M,112607,193
1,2017-18,S,None,1,10017,0,0,0,0,0,...,0,60,56,46,58,0,None,F,112607,220
2,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,1,1,3,0,None,1,112607,6
3,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,3,3,2,0,None,2,112607,9
4,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,1,2,2,0,None,3,112607,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604259,2022-23,S,None,58,72769,0,0,0,0,0,...,0,103,85,81,69,0,None,5,5838305,338
604260,2022-23,S,None,58,72769,0,0,0,0,0,...,0,10,9,10,9,0,None,6,5838305,38
604261,2022-23,S,None,58,72769,0,0,0,0,0,...,0,96,121,122,106,0,None,7,5838305,445
604262,2022-23,S,None,58,72769,0,0,0,0,0,...,0,30,29,28,24,0,None,9,5838305,111


#### Match `ReportingCategory` in `enr_mariginals` to that of `cde_dropped`.

In [57]:
# Gender
enr_marginals.loc[enr_marginals["ReportingCategory"] == "M", "ReportingCategory"] = "GN_M"  # Male
enr_marginals.loc[enr_marginals["ReportingCategory"] == "F", "ReportingCategory"] = "GN_F"  # Female
enr_marginals.loc[enr_marginals["ReportingCategory"] == "Z", "ReportingCategory"] = "GN_Z"  # Missing

# Race
enr_marginals.loc[enr_marginals["ReportingCategory"] == 0, "ReportingCategory"] = "RE_D"  # Not Reported
enr_marginals.loc[enr_marginals["ReportingCategory"] == 1, "ReportingCategory"] = "RE_I"  # American Indian or Alaska Native
enr_marginals.loc[enr_marginals["ReportingCategory"] == 2, "ReportingCategory"] = "RE_A"  # Asian
enr_marginals.loc[enr_marginals["ReportingCategory"] == 3, "ReportingCategory"] = "RE_P"  # Pacific Islander
enr_marginals.loc[enr_marginals["ReportingCategory"] == 4, "ReportingCategory"] = "RE_F"  # Filipino
enr_marginals.loc[enr_marginals["ReportingCategory"] == 5, "ReportingCategory"] = "RE_H"  # Hispanic or Latino 
enr_marginals.loc[enr_marginals["ReportingCategory"] == 6, "ReportingCategory"] = "RE_B"  # African American
enr_marginals.loc[enr_marginals["ReportingCategory"] == 7, "ReportingCategory"] = "RE_W"  # White
enr_marginals.loc[enr_marginals["ReportingCategory"] == 9, "ReportingCategory"] = "RE_T"  # Two or More Races

enr_marginals

,AcademicYear,AggregateLevel,Charter,CountyCode,DistrictCode,GR_01,GR_02,GR_03,GR_04,GR_05,...,GR_08,GR_09,GR_10,GR_11,GR_12,GR_KN,GR_TK,ReportingCategory,SchoolCode,TOTAL_ENR
0,2017-18,S,None,1,10017,0,0,0,0,0,...,0,49,53,56,35,0,None,GN_M,112607,193
1,2017-18,S,None,1,10017,0,0,0,0,0,...,0,60,56,46,58,0,None,GN_F,112607,220
2,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,1,1,3,0,None,RE_I,112607,6
3,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,3,3,2,0,None,RE_A,112607,9
4,2017-18,S,None,1,10017,0,0,0,0,0,...,0,1,1,2,2,0,None,RE_P,112607,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604259,2022-23,S,None,58,72769,0,0,0,0,0,...,0,103,85,81,69,0,None,RE_H,5838305,338
604260,2022-23,S,None,58,72769,0,0,0,0,0,...,0,10,9,10,9,0,None,RE_B,5838305,38
604261,2022-23,S,None,58,72769,0,0,0,0,0,...,0,96,121,122,106,0,None,RE_W,5838305,445
604262,2022-23,S,None,58,72769,0,0,0,0,0,...,0,30,29,28,24,0,None,RE_T,5838305,111


#### Upload to `CensusDay` table.

In [58]:
con = sqlite3.connect(DATABASE)
cur = con.cursor()

In [59]:
if DB_INSERT_SE:
    enr_marginals.to_sql("CensusDay", con, index=False, if_exists="append")

In [60]:
con.close()